In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('resume_data.csv')
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
 pd.read_csv('resume_data.csv').shape

(4880, 21)

In [ ]:
print(df.shape)

(4854, 21)


In [ ]:
job_description = """
Позиция: Системный администратор.
Обязательны знания и опыт работы с серверными операционными системами Windows Server и Linux.
Умение настраивать сетевое оборудование.
Нужен аналитический склад ума. Не стоит нам писать, если вы не готовы целый день сидеть за компьютером.
Опыт работы с системами виртуализации и облачными сервисами.
Работа с базами данных и системами резервного копирования.
"""
labels = {
    'Position': 'Позиция:',
    'Specializations': 'Специализации:',
    'Previous_Positions': 'Предыдущие позиции:',
    'About_Me': 'Обо мне:',
    'Skills': 'Навыки:'
}

text_columns = ['Position', 'Specializations', 'Previous_Positions', 'About_Me', 'Skills']

resume_texts = df[text_columns].fillna('').apply(lambda x: '. '.join([f"{labels[col]} {x[col]}" for col in text_columns if x[col].strip() != '']), axis=1).tolist()

In [ ]:
from nltk import word_tokenize
def clean_text(text):
    # Replace unwanted character sequences with a space or nothing
    text = text.replace("\n", " ").replace("\uf0d8", "")

    # Additional cleaning can be done here as needed

    return text.strip()

# Assuming resume_texts is a list of combined text entries
resume_texts = [clean_text(text) for text in resume_texts]

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask




device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#Load AutoModel from huggingface model repository
tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_mt_nlu_ru")
model = AutoModel.from_pretrained("ai-forever/sbert_large_mt_nlu_ru")

model.to(device)



BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(120138, 1024, padding_idx=0)
    (position_embeddings): Embedding(512, 1024)
    (token_type_embeddings): Embedding(2, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-23): 24 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inp

In [ ]:
torch.cuda.empty_cache()

In [ ]:
def get_embeddings(sentences):
    encoded_input = tokenizer(sentences, padding=True, truncation=True, max_length=512, return_tensors='pt').to(device)
    #Compute token embeddings

    with torch.no_grad():
        model_output = model(**encoded_input)
    #Perform pooling. In this case, mean pooling
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
    return sentence_embeddings


In [ ]:
test = get_embeddings(job_description)

In [ ]:
test.size()

torch.Size([1, 1024])

In [ ]:
tests = []
tests.append(test)

In [ ]:
tests.append(test)

In [ ]:
dict(enumerate(tests))

{0: tensor([[ 0.0316,  0.2082, -0.6679,  ...,  0.5201,  0.0377, -0.2673]],
        device='cuda:0'),
 1: tensor([[ 0.0316,  0.2082, -0.6679,  ...,  0.5201,  0.0377, -0.2673]],
        device='cuda:0')}

In [ ]:
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm


class TextDataset(Dataset):
    def __init__(self, text_list):
      self.text_list = text_list

    def __len__(self):
      return len(self.text_list)

    def __getitem__(self, idx):
      return self.text_list[idx]

dataset = TextDataset(resume_texts)
dataloader = DataLoader(dataset, batch_size=128, shuffle=False)
embeddings = torch.empty(0, 1024).to(device)
with torch.no_grad():
  for batch in tqdm(dataloader, desc="Processing batches"):
    batch_embeddings = get_embeddings(batch)
    embeddings = torch.cat((embeddings, batch_embeddings), dim=0)

print(embeddings.size())

Processing batches: 100%|██████████| 38/38 [08:19<00:00, 13.14s/it]

torch.Size([4854, 1024])


In [ ]:
job_description = """
Вакансия
Название должности: Программист C# (Middle/Senior)
Основные задачи:
1. Разработка и поддержка корпоративных приложений на платформе .NET с использованием C#;
2. Взаимодействие с командой для создания качественного и масштабируемого кода;
3. Анализ требований к продукту и разработка технических решений;
4. Проведение код-ревью и оптимизация существующего кода;
5. Участие в проектировании архитектуры проектов.

Требования к кандидату:

1. Опыт работы с C# и .NET Framework/.NET Core не менее 3 лет;
2. Знание принципов ООП, SOLID, DRY, KISS;
3. Опыт работы с базами данных (SQL и NoSQL);
4. Навыки работы с Git;
5. Опыт разработки многопоточных приложений;
6. Умение писать юнит-тесты и понимание принципов TDD;
7. Хорошие коммуникативные навыки и способность работать в команде;
8. Владение английским языком на уровне не ниже Intermediate.
"""
vacancy_embedding = get_embeddings([job_description])

In [ ]:
from torch import cosine_similarity

cosine_similarities= cosine_similarity(vacancy_embedding, embeddings).flatten()

resume_similarities_tfidf = dict(enumerate(cosine_similarities))

sorted_resume_similarities = sorted(resume_similarities_tfidf.items(), key=lambda item: item[1], reverse=True)

In [ ]:
sorted_resume_similarities[:15]

[(1300, tensor(0.8923, device='cuda:0')),
 (3614, tensor(0.8875, device='cuda:0')),
 (3592, tensor(0.8870, device='cuda:0')),
 (1062, tensor(0.8828, device='cuda:0')),
 (2127, tensor(0.8828, device='cuda:0')),
 (2195, tensor(0.8828, device='cuda:0')),
 (4743, tensor(0.8805, device='cuda:0')),
 (335, tensor(0.8801, device='cuda:0')),
 (2894, tensor(0.8794, device='cuda:0')),
 (779, tensor(0.8787, device='cuda:0')),
 (3745, tensor(0.8753, device='cuda:0')),
 (3013, tensor(0.8746, device='cuda:0')),
 (4046, tensor(0.8746, device='cuda:0')),
 (3829, tensor(0.8744, device='cuda:0')),
 (237, tensor(0.8733, device='cuda:0'))]

In [ ]:
sorted_df = df.iloc[[index for index, similarity in sorted_resume_similarities]]

sorted_df.head(15)


,Position,Sex,Age,City,Is_Relocation_Ready,Is_Business_Trip_Ready,Specializations,Employment,Work_Schedule,Desired_Salary,...,Skills,Languages,Education,Courses,Has_Car,Driving_Categories,About_Me,Job_Search_Status,URL,Embeddings
1300,Ведущий разработчик,Мужчина,39.0,Алматы,False,True,"Программист, разработчик, Архитектор, Руководи...","полная занятость, частичная занятость, проектн...","полный день, сменный график, гибкий график, уд...",NaN,...,"Cистемы управления базами данных, Joomla CMS, ...","Казахский - Родной, Английский - B1",Алматинский институт энергетики и связи - Ради...,NaN,False,NaN,"Разработка ПО, БД и СУБД. Внедрение, сопровожд...","Предложили работу, решает",https://spb.hh.ru/resume/8e7076e000015804ff003...,"[0.550189733505249, 0.298458456993103, -1.3737..."
3614,Программист ERP систем 1С / удаленная работа,Мужчина,47.0,Щелково,False,True,"Программист, разработчик",полная занятость,удаленная работа,250000.0,...,Обучаемость,"Русский - Родной, Английский - B2",Томский государственный архитектурно строитель...,"Учебный центр ""Специалист"" при МГТУ им. Н.Э. Б...",False,NaN,"Имею комплексный подход к задачам, аналитическ...",Активно ищет работу,https://spb.hh.ru/resume/75121359000178d9de003...,"[0.6192873120307922, 0.24165144562721252, -0.9..."
3592,ИТ-специалист,Мужчина,36.0,Жуковский,False,True,Специалист технической поддержки,"полная занятость, проектная работа","полный день, сменный график, удаленная работа",120000.0,...,"Информационные технологии, Техническая поддерж...","Русский - Родной, Английский - A1","Московский энергетический институт, Москва - К...",NaN,True,B,Опыт администрирования и обеспечения работоспо...,Рассматривает предложения,https://spb.hh.ru/resume/48d3366d0004655d45003...,"[0.3672751784324646, 0.21559642255306244, -1.0..."
1062,.NET Developer,Мужчина,35.0,Минск,False,True,"Программист, разработчик","полная занятость, частичная занятость","полный день, гибкий график, удаленная работа",NaN,...,"C#, .NET Framework, WPF, Windows Forms, MS SQL...","Русский - Родной, Английский - B1","Филиал Компьютерной академии ""ШАГ"", Минск (Бел...",NaN,False,B,Готов выполнить Ваше тестовое задание.\n\nУров...,Активно ищет работу,https://spb.hh.ru/resume/c6ccc2af0003d07920003...,"[0.4485098123550415, 0.053186796605587006, -0...."
2127,.NET Developer,Мужчина,35.0,Минск,False,True,"Программист, разработчик","полная занятость, частичная занятость","полный день, гибкий график, удаленная работа",NaN,...,"C#, .NET Framework, WPF, Windows Forms, MS SQL...","Русский - Родной, Английский - B1","Филиал Компьютерной академии ""ШАГ"", Минск (Бел...",NaN,False,B,Готов выполнить Ваше тестовое задание.\n\nУров...,Активно ищет работу,https://spb.hh.ru/resume/c6ccc2af0003d07920003...,"[0.4485098123550415, 0.053186796605587006, -0...."
2195,.NET Developer,Мужчина,35.0,Минск,False,True,"Программист, разработчик","полная занятость, частичная занятость","полный день, гибкий график, удаленная работа",NaN,...,"C#, .NET Framework, WPF, Windows Forms, MS SQL...","Русский - Родной, Английский - B1","Филиал Компьютерной академии ""ШАГ"", Минск (Бел...",NaN,False,B,Готов выполнить Ваше тестовое задание.\n\nУров...,Активно ищет работу,https://spb.hh.ru/resume/c6ccc2af0003d07920003...,"[0.4485098123550415, 0.053186796605587006, -0...."
4743,Системный администратор,Мужчина,41.0,Белгород,True,False,"Системный администратор, Системный инженер, Сп...","полная занятость, частичная занятость, проектн...","полный день, сменный график, гибкий график, уд...",50000.0,...,"Сборка ПК, Ремонт ПК, Закупка оргтехники и обо...","Русский - Родной, Английский - B2",Санкт-Петербургский государственный политехнич...,NaN,True,B,**********************************************...,Активно ищет работу,https://spb.hh.ru/resume/438368430007bc6c43003...,"[0.17014414072036743, 0.26617586612701416, -0...."
335,Разработчик,Мужчина,38.0,Москва,False,True,"Руководитель группы разработки, Программист, р...",полная занятость,"полный день, удаленная работа",NaN,...,"XML, Vi

In [ ]:
assert len(df) == embeddings.shape[0]


In [ ]:
embeddings.size()
embeddings_list = embeddings.cpu().numpy().tolist()  # Move embeddings back to CPU and convert to list

# Add embeddings as a new column in the DataFrame
df['Embeddings'] = embeddings_list

In [ ]:
df.head()

,Position,Sex,Age,City,Is_Relocation_Ready,Is_Business_Trip_Ready,Specializations,Employment,Work_Schedule,Desired_Salary,...,Skills,Languages,Education,Courses,Has_Car,Driving_Categories,About_Me,Job_Search_Status,URL,Embeddings
0,Системный администратор,Мужчина,40.0,Колпино,False,True,Системный администратор,"частичная занятость, проектная работа","сменный график, гибкий график, удаленная работа",NaN,...,Windows Server 2003/2008R2/2012R2 на уровне ад...,"Русский - Родной, Английский - A1",Российский государственный гидрометеорологичес...,"Mikrotik MTCNA - SPW, Корпоративная IP-телефон...",True,B,Сертификаты от Microsoft: MTA: Windows Server®...,Рассматривает предложения,https://spb.hh.ru/resume/12275fe400043bd2ee003...,"[0.32768160104751587, 0.08176106214523315, -0...."
1,Администратор-кассир,Женщина,22.0,Екатеринбург,True,True,"Оператор ПК, оператор базы данных, Делопроизво...","полная занятость, частичная занятость","сменный график, гибкий график",45000.0,...,"Пользователь Autocad, Arhicad, Пользователь wo...",Русский - Родной,УКТП - Строительство и эксплуатация зданий и с...,NaN,False,NaN,Быстро учусь. Ответственно подхожу к работе,Активно ищет работу,https://spb.hh.ru/resume/c3ad30c90007195c21003...,"[0.345800518989563, 0.1473490446805954, -1.682..."
2,"Производственный руководитель, начальник участ...",Мужчина,38.0,Москва,True,True,"Начальник производства, Начальник смены, масте...","полная занятость, частичная занятость, проектн...","полный день, сменный график, гибкий график, уд...",NaN,...,"Коммуникабельность, Мобильность, Пунктуальност...","Русский - Родной, Английский - A1, Китайский - A1",Алма атинская Академия Бизнеса - Управление пе...,NaN,False,B,Азартен в работе. Не приемлю полумер. Увлекаюс...,Рассматривает предложения,https://spb.hh.ru/resume/9f3bb5f70001fae21e003...,"[1.0071132183074951, 0.1831478774547577, -1.17..."
3,Специалист по охране окружающей среды (эколог),Женщина,33.0,Новый Уренгой,True,True,Инженер по охране труда и технике безопасности...,"полная занятость, частичная занятость","полный день, сменный график, гибкий график, уд...",NaN,...,"Проведение тренинга для новых сотрудников, Орг...","Русский - Родной, Английский - B2",ООО Образовательный центр «Техноимпульс» - Упр...,Обеспечение экологической безопасности руковод...,False,B,"Я трудолюбивый и ответственный человек, требов...",Активно ищет работу,https://spb.hh.ru/resume/c730fa57000435d95e003...,"[0.21396110951900482, 0.12703493237495422, -1...."
4,"Оператор 1с, Товаровед 1с",Женщина,38.0,Красноярск,False,False,"Оператор ПК, оператор базы данных",полная занятость,полный день,50000.0,...,"1С: Торговля и склад, Складской Учет, ТТН, Вво...",Русский - Родной,Красноярский Монтажный колледж - Экономист,"ЧУП""ЭЛИТА СЕРВИС"" - ЧУП""ЭЛИТА СЕРВИС"", маникюр...",False,B,"Приятная внешность, коммуникабельная,ответстве...",Активно ищет работу,https://spb.hh.ru/resume/79b35d240000eb1a51003...,"[0.4720255434513092, 0.23111078143119812, -1.5..."


In [ ]:
df.to_csv('resume_data_n_embeddings.csv')

KeyboardInterrupt: ignored

In [ ]:
df.loc[1062]

Position                                                     .NET Developer
Sex                                                                 Мужчина
Age                                                                    35.0
City                                                                  Минск
Is_Relocation_Ready                                                   False
Is_Business_Trip_Ready                                                 True
Specializations                                    Программист, разработчик
Employment                            полная занятость, частичная занятость
Work_Schedule                  полный день, гибкий график, удаленная работа
Desired_Salary                                                          NaN
Total_Experience                                                      10.83
Previous_Positions        Инженер-программист, Инженер-программист, Инже...
Skills                    C#, .NET Framework, WPF, Windows Forms, MS SQL...
Languages   